In [ ]:
import pandas as pd
import numpy as np
import pymysql
import pickle as pkle

- DB에서 데이터프레임 형태로 추출하기

In [ ]:
# mysql 연동
conn = pymysql.connect(host='127.0.0.1'
                , port=3306
                , user = 'root'
                , password='root1234'
                , db = 'movie_pj'
                , charset='utf8'
                ) 

cursor = conn.cursor()

In [ ]:
movies_sql = 'SELECT * FROM movies'
movies = pd.read_sql(movies_sql, conn)
movies.head()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_1536\483360329.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies = pd.read_sql(movies_sql, conn)


,movie_id,title,release_date,imdb_url,all_genres
0,0,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,Animation-Children-Comedy
1,1,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,Action-Adventure-Thriller
2,2,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,Thriller
3,3,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,Action-Comedy-Drama
4,4,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),Crime-Drama-Thriller


In [ ]:
users_sql = 'SELECT * FROM users'
users = pd.read_sql(users_sql, conn)
users.head()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_1536\2420501795.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql(users_sql, conn)


,user_id,age,sex,occupation
0,0,24,M,technician
1,1,53,F,other
2,2,23,M,writer
3,3,24,M,technician
4,4,33,F,other


In [ ]:
ratings_sql = 'SELECT * FROM ratings'
ratings = pd.read_sql(ratings_sql, conn)
ratings.head()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_1536\890323743.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql(ratings_sql, conn)


,rating_id,user_id,movie_id,rating
0,2,195,241,3
1,3,185,301,3
2,4,21,376,1
3,5,243,50,2
4,6,165,345,1


In [ ]:
conn.commit()
conn.close()

----

- train, test set

In [ ]:
# df를 train/test set으로 분할하는 함수 생성
# Args: df, holdoyt_fraction(분할 비율)
def split_dataframe(df, holdout_fraction=0.1):
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]

  return train, test

In [ ]:
ratings['user_id'] = ratings['user_id'].astype(int)
ratings['movie_id'] = ratings['movie_id'].astype(int)

train_ratings, test_ratings = split_dataframe(ratings)
train_ratings.shape, test_ratings.shape

((90000, 4), (10000, 4))

### Matrix Factorization

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, dot

- Modeling

In [ ]:
users.shape[0], movies.shape[0]

(943, 1682)

In [ ]:
# n_latent_factors = 30 # user와 movie embedding의 차원수

# user_input = Input(shape=[1], name='user')
# movie_input = Input(shape=[1], name='movie')

# user_embedding = Embedding(input_dim=users.shape[0] # 943
#                            , output_dim = n_latent_factors # 30
#                            , name='user_embedding'
#                            )(user_input)

# movie_embedding = Embedding(input_dim=movies.shape[0] # 1682
#                            , output_dim = n_latent_factors # 30
#                            , name='movie_embedding'
#                            )(movie_input)

# user_vec = Flatten(name='flatten_users')(user_embedding) # 1차원 배열로 변환
# movie_vec = Flatten(name='flatten_movies')(movie_embedding) # 1차원 배열로 변환

# product = dot([movie_vec, user_vec], axes=1) # 평점
# model = Model(inputs=[user_input, movie_input], outputs=product) # user와 movie가 주어졌을 때, 평점을 예측하는 모델

# model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 movie (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 user (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 movie_embedding (Embedding)    (None, 1, 30)        50460       ['movie[0][0]']                  
                                                                                                  
 user_embedding (Embedding)     (None, 1, 30)        28290       ['user[0][0]']                   
                                                                                            

In [ ]:
# model.compile(optimizer='adam', loss='mse')

In [ ]:
# history = model.fit(x=[train_ratings['user_id'], train_ratings['movie_id']],
#                     y=train_ratings['rating'], epochs=500,
#                     validation_data=([test_ratings['user_id'], test_ratings['movie_id'] ],
#                                      test_ratings['rating']),
#                     verbose=1, batch_size=train_ratings.shape[0])

In [ ]:
# model.save('CFModel01.h5')

- Develop

In [54]:
#@title Solution
DOT = 'dot'
COSINE = 'cosine'
def compute_scores(query_embedding, item_embeddings, measure=DOT):
  """Computes the scores of the candidates given a query.
  Args:
    query_embedding: a vector of shape [k], representing the query embedding.
    item_embeddings: a matrix of shape [N, k], such that row i is the embedding
      of item i.
    measure: a string specifying the similarity measure to be used. Can be
      either DOT or COSINE.
  Returns:
    scores: a vector of shape [N], such that scores[i] is the score of item i.
  """
  u = query_embedding
  V = item_embeddings
  if measure == COSINE:
    V = V / np.linalg.norm(V, axis=1, keepdims=True)
    u = u / np.linalg.norm(u)
  scores = u.dot(V.T)
  return scores

In [55]:
# @title User recommendations and nearest neighbors (run this cell)
def user_recommendations(model, measure=DOT, exclude_rated=False, k=6, user_id=570):
  if USER_RATINGS:
    scores = compute_scores(
        model.embeddings["user_id"][user_id], model.embeddings["movie_id"], measure)
    score_key = measure + ' score'
    df = pd.DataFrame({
        score_key: list(scores),
        'movie_id': movies['movie_id'],
        'titles': movies['title'],
        'genres': movies['all_genres'],
    })
    if exclude_rated:
      # remove movies that are already rated
      rated_movies = ratings[ratings.user_id == str(user_id)]["movie_id"].values
      df = df[df.movie_id.apply(lambda movie_id: movie_id not in rated_movies)]
    return (df.sort_values([score_key], ascending=False).head(k))

In [56]:
import tensorflow as tf 
CFModel01 = tf.keras.models.load_model('data/CFModel01.h5', compile=False)
CFModel01.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 movie (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 user (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  


 movie_embedding (Embedding)    (None, 1, 30)        50460       ['movie[0][0]']                  
                                                                                                  
 user_embedding (Embedding)     (None, 1, 30)        28290       ['user[0][0]']                   
                                                                                                  
 flatten_movies (Flatten)       (None, 30)           0           ['movie_embedding[0][0]']        
                                                                                                  
 flatten_users (Flatten)        (None, 30)           0           ['user_embedding[0][0]']         
                                                                                                  
 dot (Dot)                      (None, 1)            0           ['flatten_movies[0][0]',         
                                                                  'flatten_users[0][0]']          
          

In [57]:
CFModel01.embeddings = {
    'user_id': CFModel01.get_layer('user_embedding').weights[0].numpy(), # U (943, 30)
    'movie_id': CFModel01.get_layer('movie_embedding').weights[0].numpy() # V (1682, 30)
}

In [58]:
USER_RATINGS = True

In [59]:
result_df = user_recommendations(CFModel01, measure=DOT, k=10, user_id=600, exclude_rated=True)

In [60]:
result_df

,dot score,movie_id,titles,genres
813,4.143117,813,"Great Day in Harlem, A (1994)",Documentary
1642,4.132014,1642,Angel Baby (1995),Drama
850,4.065010,850,Two or Three Things I Know About Her (1966),Drama
1448,4.060930,1448,Pather Panchali (1955),Drama
1466,4.048428,1466,"Saint of Fort Washington, The (1993)",Drama
1641,4.039691,1641,Some Mother's Son (1996),Drama
1598,4.010055,1598,Someone Else's America (1995),Drama
1499,4.001060,1499,Santa with Muscles (1996),Comedy
1397,3.959460,1397,Anna (1996),Drama
1305,3.959085,1305,Delta of Venus (1994),Drama
